In [3]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

repo_root = Path.cwd().parent  # docs/ -> repo root
sys.path.insert(0, str(repo_root))
# import tests.fixtures.test_case_catalog as cases
import tests.fixtures.test_case_catalog as cases



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# single sim test case

In [ ]:
case = cases.Local_cases.retrieve_norfolk_single_sim_test_case(
        start_from_scratch=False, download_if_exists=False
    )


In [5]:
case.system.cfg_system.display_tabulate_cfg()

+---------------------------+--------------------------------------------------------------+--------------------------------------------------------------+
| attr_name                 | desc                                                         | val                                                          |
+===========================+==============================================================+==============================================================+
| system_directory          | Path where TRITON-SWMM system outputs will be stored.        | /home/dcl3nd/dev/TRITON-                                     |
|                           |                                                              | SWMM_toolkit/test_data/norfolk_coastal_flooding/tests        |
+---------------------------+--------------------------------------------------------------+--------------------------------------------------------------+
| watershed_gis_polygon     | Watershed or subcatchment gis used

In [6]:
case.analysis.cfg_analysis.print_files_defined_in_yaml()

/home/dcl3nd/
├── dev/
│   └── TRITON-SWMM_toolkit/
│       └── test_data/
│           └── norfolk_coastal_flooding/
│               └── tests/
│                   └── single_sim/
│                       ├── weather_tseries.nc
│                       └── weather_indices.csv
└── .local/
    └── share/
        └── TRITON_SWMM_toolkit/
            └── examples/
                └── norfolk_coastal_flooding/
                    └── data/
                        └── contents/
                            └── shp_boundary_condition/
                                └── variable_bc.shp


In [7]:
case.system.process_system_level_inputs()

In [8]:
case.system.compile_TRITON_SWMM(redownload_triton_swmm_if_exists=False, recompile_if_already_done_successfully=False)

[CMAKE PATCH] cmake/machine.cmake already patched (skipping)

Compiling CPU Backend
[CPU] Starting compilation...
[CPU]   Script: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/compile_cpu.sh
[CPU] ✓ Compilation successful!

Compilation Status
✓ CPU backend: COMPILED SUCCESSFULLY
  Build: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/triton/build_cpu
  GPU backend: NOT REQUESTED

Available backends: cpu



In [9]:
case.analysis.run_prepare_scenarios_serially(
        overwrite_scenarios=True, rerun_swmm_hydro_if_outputs_exist=True
    )

2026-01-30 16:18:18,332 [INFO] Loading system configuration from /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/tests.yaml
2026-01-30 16:18:18,335 [INFO] Loading analysis configuration from /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/single_sim/single_sim.yaml
2026-01-30 16:18:18,341 [INFO] Preparing scenario 0
[Scenario 0] Using CPU backend

... SWMM Version 5.2.4
 o  Retrieving project data
 o  Simulating day: 0     hour: Simulation complete           
... Run CompleteScenario preparation complete
2026-01-30 16:18:21,566 [INFO] Scenario 0 prepared successfully


In [10]:
case.analysis.run_sims_in_sequence(pickup_where_leftoff=False)

In [11]:
case.analysis.process_all_sim_timeseries_serially()

Processing run results for scenario 0
Processed TRITON outputs for scenario 0
##################################
Found problem. Orifice conduits do not return max velocity or max over full flow. Filling with empty string
Normal row vs. problem row:
  0                    CONDUIT     0.000     0  00:01      0.00    0.00    0.00

  756                  ORIFICE     0.000     0  00:00                      0.00

Properly parsed values:
['756', 'ORIFICE', '0.000', '0', '00:00', '', '', '0.00\n']
Converted variable to datatype = type, <class 'str'>
Converted variable to datatype = type, <class 'str'>
Processed SWMM outputs for scenario 0


# multi-sim case

In [ ]:
case = cases.Local_TestCases.retrieve_norfolk_multi_sim_test_case(start_from_scratch=False)
analysis = case.analysis

In [13]:
# analysis = norfolk_multi_sim_analysis

result = analysis.submit_workflow(
    mode="local",
    process_system_level_inputs=True,
    overwrite_system_inputs=True,
    compile_TRITON_SWMM=True,
    recompile_if_already_done_successfully=True,
    prepare_scenarios=True,
    overwrite_scenario=True,
    rerun_swmm_hydro_if_outputs_exist=True,
    process_timeseries=True,
    which="both",
    clear_raw_outputs=True,
    overwrite_if_exist=True,
    compression_level=5,
    pickup_where_leftoff=False,
    dry_run=False,
    verbose=True,
)

[Snakemake] Submitting workflow in local mode
[Snakemake] Snakefile generated: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/Snakefile
[Snakemake] Running workflow locally with Snakemake
[Snakemake] DRY RUN
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/.snakemake_profile/local


[Snakemake] Workflow completed successfully
[Snakemake] Running workflow locally with Snakemake
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/.snakemake_profile/local
[Snakemake] Workflow completed successfully


In [9]:
analysis.SWMM_node_summary

ValueError: could not open file because it does not exist: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/SWMM_nodes.zarr. Run analysis.consolidate_[SWMM/TRITON]_outputs().

# sensitivity analysis

In [ ]:
nrflk_cpu_sensitivity = cases.Local_TestCases.retrieve_norfolk_cpu_config_sensitivity_case(
        start_from_scratch=True
    )
analysis = nrflk_cpu_sensitivity.analysis

In [6]:
result = analysis.submit_workflow(
    mode="local",
    process_system_level_inputs=True,
    overwrite_system_inputs=True,
    compile_TRITON_SWMM=True,
    recompile_if_already_done_successfully=True,
    prepare_scenarios=True,
    overwrite_scenario=True,
    rerun_swmm_hydro_if_outputs_exist=True,
    process_timeseries=True,
    which="both",
    clear_raw_outputs=True,
    overwrite_if_exist=True,
    compression_level=5,
    pickup_where_leftoff=False,
    dry_run=True,
    verbose=True,
)

[Snakemake] Submitting sensitivity analysis workflow in local mode
[Snakemake] Generated master Snakefile: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/cpu_config_sensitivity/Snakefile
[Snakemake] Created required directories (_status, logs/sims)
[Snakemake] Running workflow locally with Snakemake
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/cpu_config_sensitivity/.snakemake_profile/local
[Snakemake] Workflow completed successfully


In [ ]:
case = cases.Local_TestCases.retrieve_norfolk_cpu_config_sensitivity_case(start_from_scratch=False)
analysis = case.analysis

In [ ]:
analysis.sensitivity.compile_TRITON_SWMM_for_sensitivity_analysis()
assert analysis.sensitivity.compilation_successful == True

In [ ]:
analysis.sensitivity.prepare_scenarios_in_each_subanalysis(concurrent=True)
assert analysis.log.all_scenarios_created.get() == True

In [ ]:
analysis.sensitivity.run_all_sims(pickup_where_leftoff=False, concurrent=False)
# assert analysis.log.all_sims_run.get() == True